[This notebook describes the behavior in Julia 0.5 and later. There were some significant differences in earlier versions of Julia that are not described here.]

In [1]:
M = rand(3, 3)

3×3 Array{Float64,2}:
 0.481986  0.994296  0.29149 
 0.688254  0.400389  0.826445
 0.77697   0.548805  0.527509

In [2]:
A = view(M, 1:2, 1:2)

2×2 SubArray{Float64,2,Array{Float64,2},Tuple{UnitRange{Int64},UnitRange{Int64}},false}:
 0.481986  0.994296
 0.688254  0.400389

##  Defining new types

Types form one of the cornerstones of Julia, enabling both its elegance and speed (despite the fact that one usually explicitly refers to types much less than in, say, C++).

In this notebook, we will look at how to define new types in Julia. This is very common when you need a certain **kind (type) of behaviour** that is not captured by the "built-in" types.

User-defined types are on the same footing as the "built-in" types in the standard library (`Int`, `Float64`, etc.), which are themselves defined in Julia's base library (which is written in Julia). The compiler is often able to "optimize away" the type structure, so that the resulting code is just as efficient as it would have been if you had not used types and had just written out the corresponding operations. We are thus at liberty to use types to structure our code and reflect the model of the world that we wish to implement, without suffering from poor performance.

## Basic type definitions

We will look at a concrete example that arose naturally in real code in the author's pacakage 
[`IntervalConstraintProgramming.jl`](https://github.com/dpsanders/IntervalConstraintProgramming.jl): defining a type  to represent the volume of an object. This is similar to a [post on the `julia-users` mailing list](https://groups.google.com/forum/#!searchin/julia-users/probability/julia-users/PpXLxHajsfA/UWpmJZd2BQAJ) about defining a type to represent a probability.  

Despite the fact that the type contains only a single field, many of the confusing points about types in Julia can be best appreciated in this apparently simple situation.

We could choose to represent a volume with a standard `Float64`, but, as we shall see, there are good reasons to wrap this into a new type:

In [4]:
type Vol
    value::Float64
end

[The keyword `type` is likely to change in a future version of Julia.] 

This defines a "template" for making objects (which are basically boxes that contain information), and specifies their internal structure; the resulting structure / shape is given the name `Vol`. Each is specified to contain a single object, a variable with the name `value` of type `Float64`. Note, however, that no actual object has yet been created.

To create an object of type `Vol`, we write

In [5]:
V = Vol(3)

Vol(3.0)

Note that this **constructor** (a function with the same name as the type that creates objects of that type) was **automatically generated** when we defined the type. We can ask Julia which of these so-called default constructors are provided:

In [6]:
methods(Vol)

# 3 methods for generic function "(::Type)":
Vol(value::Float64) at In[4]:2
Vol(value) at In[4]:2
(::Type{T}){T}(arg) at sysimg.jl:53

In [7]:
W = Vol(10.5)

Vol(10.5)

Trying to use 

In [8]:
Vol(true)

Vol(1.0)

In [9]:
Vol("David")

LoadError: LoadError: MethodError: Cannot `convert` an object of type String to an object of type Float64
This may have arisen from a call to the constructor Float64(...),
since type constructors fall back to convert methods.
while loading In[9], in expression starting on line 1

gives an error, due to 

In [10]:
convert(Float64, "David")

LoadError: LoadError: MethodError: Cannot `convert` an object of type String to an object of type Float64
This may have arisen from a call to the constructor Float64(...),
since type constructors fall back to convert methods.
while loading In[10], in expression starting on line 1

In [12]:
convert(Float64, "3.1")

LoadError: LoadError: MethodError: Cannot `convert` an object of type String to an object of type Float64
This may have arisen from a call to the constructor Float64(...),
since type constructors fall back to convert methods.
while loading In[12], in expression starting on line 1

In [13]:
parse(Float64, "3.1")

3.1

In [16]:
workspace()

In [17]:
import Base.convert

convert(::Type{Float64}, s::String) = parse(Float64, s)

convert (generic function with 600 methods)

In [19]:
type Vol
    value::Float64
end

In [20]:
Vol("David")

LoadError: LoadError: ArgumentError: invalid number format "David" for Float64
while loading In[20], in expression starting on line 1

In [23]:
Vol("3.5")

Vol(3.5)

To interactively introspect and find out whath is inside the object `V`, we write `V.<TAB>`[press the `TAB` key after typing `V.` at the REPL, in the notebook, or in Atom].

Alternatively, we can use `fieldnames`, which returns an array of `Symbol`s, containing the names of the fields:

In [ ]:
fieldnames(V)

We access the field with any of the following commands:

In [ ]:
V.value

In [ ]:
getfield(V, :value)  # by name

In [ ]:
getfield(V, 1)       # by number

In a `type`, we can also change the values of the variables in the object

In [25]:
V = Vol(3.5)

Vol(3.5)

In [26]:
V.value = 10


10

In [27]:
V

Vol(10.0)

We may want to prevent a user from altering the value stored in the object. To do this, use `immutable` instead of `type`:

In [34]:
workspace()

In [36]:
immutable Vol2
    value::Float64
    a::Vector{Int64}
end

In [37]:
V2 = Vol2(3.5, [10])

Vol2(3.5,[10])

In [38]:
push!(V2.a, 13)

2-element Array{Int64,1}:
 10
 13

In [39]:
V2

Vol2(3.5,[10,13])

In [30]:
V2.value

3.5

In [31]:
V2.value = 10

LoadError: LoadError: type Vol2 is immutable
while loading In[31], in expression starting on line 1

## Constructors

So far, we have been using the default constructors. These have the following problem:

In [41]:
type Vol
    value::Float64
end

In [42]:
Vol(-1)

Vol(-1.0)

One (bad) solution:

In [43]:
vol(x::Float64) = x < 0 ? error("Negative") : Vol(x)

vol (generic function with 1 method)

In [45]:
vol(10.0)

Vol(10.0)

In [47]:
vol(-1.0)

LoadError: LoadError: Negative
while loading In[47], in expression starting on line 1

In [48]:
Vol(-1.0)

Vol(-1.0)

This should not be allowed, since volumes cannot be negative. We want to impose a constraint *at the moment when we  create the object*, i.e. in the constructor. To do so, we define an **inner constructor**, i.e. a constructor that is defined *inside* the type definition.

Since Julia does not allow redefining types, we need to clear the workspace before we do so:

In [49]:
workspace()

In [50]:
immutable Vol
    value::Float64
    
    function Vol(x)
        isa(x, Real) || throw(ArgumentError("Volume must be real."))
        x < 0 && throw(ArgumentError("Negative volume not allowed."))
        
        new(x)
    end
end

In [53]:
Vol("hello")

LoadError: LoadError: ArgumentError: Volume must be real.
while loading In[53], in expression starting on line 1

In [54]:
Vol(-3)

LoadError: LoadError: ArgumentError: Negative volume not allowed.
while loading In[54], in expression starting on line 1

In [56]:
V = Vol(1)

Vol(1.0)

In [60]:
methods(Vol)

# 2 methods for generic function "(::Type)":
Vol(x) at In[50]:5
(::Type{T}){T}(arg) at sysimg.jl:53

In [67]:
workspace()

In [68]:
type Vol3
    value::Float64
end

In [62]:
methods(Vol3)

# 3 methods for generic function "(::Type)":
Vol3(_::Float64) at In[61]:2
Vol3(_) at In[61]:2
(::Type{T}){T}(arg) at sysimg.jl:53

In [69]:
V = Vol3(3.0)

Vol3(3.0)

In [70]:
Vol3(V::Vol3) = Vol3(V.value)

Vol3

In [74]:
W = Vol3(V)

Vol3(3.0)

In [75]:
a = [1 ,2]

2-element Array{Int64,1}:
 1
 2

In [77]:
b = copy(a)

2-element Array{Int64,1}:
 1
 2

Here we have used so-called ["short-circuit" evaluation](http://docs.julialang.org/en/release-0.4/manual/control-flow/#man-short-circuit-evaluation): `&&` can be thought of as an `if...then`, while `||` is `unless...` (i.e. `if not ... then`).

The function `new` is a special function that instantiates (creates) the object. Using `new`, some of the fields may be left empty, by not including them in the call to `new`, and filled in later (if the type is not `immutable`), for example:

In [81]:
workspace()

type Example
    a::Float64
    b::Vector{Int}
    
    Example(a) = new(a)
end

Which constructors are now available?

In [80]:
methods(Example)

# 2 methods for generic function "(::Type)":
Example(a) at In[79]:7
(::Type{T}){T}(arg) at sysimg.jl:53

We see that defining our own inner constructor prevented Julia from automatically defining any constructors.

In [82]:
x = Example(3)

Example(3.0,#undef)

Here, `x.b` was not initialized. Accessing it produces an error:

In [ ]:
x.b

But we can modify its value:

In [ ]:
x.b = [1, 2]

x

Note that the constructor that accepts both arguments is not defined:

In [ ]:
Example(1, [1, 2])

If we wish to define it, then we need to add it as another inner constructor:

In [83]:
workspace()

type Example
    a::Float64
    b::Vector{Int}
    
    Example(a) = new(a)
    Example(a, b) = new(a, b)
end

In [94]:
E = Example(1, [1, 2])

Example(1.0,[1,2])

In [90]:
Example(E::Example) = Example(E.a, copy(E.b))

Example

In [95]:
E2 = Example(E)

Example(1.0,[1,2])

In [96]:
push!(E2.b, 10)

3-element Array{Int64,1}:
  1
  2
 10

In [97]:
E

Example(1.0,[1,2])

## Making types parametric

There is no reason why volumes should be `Float64` -- they may be any kind of number. We could just remove the type annotation:

In [103]:
workspace()

In [104]:
immutable Vol
    value   # DO NOT DO THIS!
end

In [ ]:
Vol(3.5)

In [ ]:
Vol(3)

In [105]:
Vol("hello")

Vol("hello")

but we should **never do this**.  [By "never", we of course mean "only if you are really sure that you know what you're doing" and won't complain about the consequences.] The reason for this is that Julia is unable to infer the type of the variable, and so will be **very slow**.

Instead, what we really mean is that there should be a *different type* of `Vol` for each type that it could contain - integers, floats, `BigFloat`s, rationals, intervals, etc.
To tell Julia this, we use a **type parameter**, written with curly braces:

In [110]:
workspace()

In [111]:
immutable Vol{T}
    value::T   # DO THIS INSTEAD!
end

In [112]:
Vol(3)

Vol{Int64}(3)

In [113]:
Vol(3.5)

Vol{Float64}(3.5)

In [114]:
Vol("Hello")

Vol{String}("Hello")

In [115]:
Vol(Vol(3))

Vol{Vol{Int64}}(Vol{Int64}(3))

In [116]:
Vol(s::String) = Vol(length(s))

Vol{T}

In [117]:
Vol("hello")

Vol{Int64}(5)

This says "for **any** type `T`, define a `Vol` with type parameter `T` as follows". This makes `Vol` a **parametric type**, a kind of meta-template (a template for templates). This turns out to be a surprisingly powerful concept when combined with multiple dispatch.  

The curly braces here, `{T}`, can be read as "for a given `T`, define `Vol`, parameterized with that `T`, as follows"; note that `T` is used in the body of the type definition. 

Again, default constructors are automatically provided:

In [ ]:
Vol{Int64}(3)

In [ ]:
Vol{Float64}(3.5)

The parameter `T` in the type definition means that when Julia sees *any* constructor called as `Vol{T}` for a given type `T`, it uses the body of the definition with `T` substituted with the corresponding type.

However, we are doing extra work that should be the job of the compiler: the compiler should be able to **infer** that `3` is an `Int64`, and indeed it can:

In [ ]:
Vol(3)

In [ ]:
Vol(3.5)

In [ ]:
Vol("hello")

In [ ]:
?<:

Here we find a new problem: the volume should be a real number. We can specify this restriction using the **subtype operator**, `<:`:

In [ ]:
workspace()

In [ ]:
Int <: Real   # "is a subtype"

In [ ]:
String <: Real

In [ ]:
immutable Vol{T <: Real}  # only accept types T that are subtypes of Real
    value::T   
end

In [ ]:
Vol("hello")

In [ ]:
Vol(3)

Let's consider a more complicated example, representing a particle in one dimension. The particle has a position and a velocity, and we would like to restrict them to have the same, real type:

In [121]:
type Particle{T <: Real}
    x::T
    v::T
end

LoadError: LoadError: invalid redefinition of constant Particle
while loading In[121], in expression starting on line 1

In [122]:
Particle("hello", "David")

LoadError: LoadError: MethodError: no method matching Particle{T<:Real}(::String, ::String)
Closest candidates are:
  Particle{T<:Real}{T}(::Any) at sysimg.jl:53
while loading In[122], in expression starting on line 1

In [119]:
p = Particle(3, 4)

Particle{Int64}(3,4)

In [120]:
p2 = Particle(3, 4.5)

LoadError: LoadError: MethodError: no method matching Particle{T<:Real}(::Int64, ::Float64)
Closest candidates are:
  Particle{T<:Real}{T<:Real}(::T<:Real, !Matched::T<:Real) at In[118]:2
  Particle{T<:Real}{T}(::Any) at sysimg.jl:53
while loading In[120], in expression starting on line 1

We see that Julia is doing something quite sophisticated: it tries to fill in the types `T` *and* checks that types with the same parameter `T` do actually match. If they do, then it generates the corresponding type with the corresponding `T`.

What if we wanted to allow `Particle(3, 4.5)`, such that both got promoted to a common type? We do this by adding a new constructor:

In [ ]:
Particle(a::Real, b::Real) = Particle(promote(a, b)...)

In [ ]:
Particle(3, 4.5)

In [123]:
promote(3, 4.5)

(3.0,4.5)

Here, the `...` operator is called a **splat**. It takes the tuple that is the result of `promote` and "undoes it", passing the values inside the tuple as individual arguments to the function `Particle`.
Without the splat, `Particle(promote(a, b))` would pass the tuple that is the result of `promote(a, b)` to the constructor `Particle` as a *single*, tuple argument.

**Exercise**: write a constructor of the `Particle` type that accepts a tuple.

## Inner constructors for parametric types

What happens if we try to add an inner constructor to a parametric type?

In [124]:
workspace()

In [125]:
immutable Vol{T<:Real}
    value::T   
    
    function Vol(x::Real)
        x < 0 && throw(ArgumentError("Negative volume not allowed."))
        
        new(x)
    end
end

In [126]:
Vol(3)

LoadError: LoadError: MethodError: Cannot `convert` an object of type Int64 to an object of type Vol{T<:Real}
This may have arisen from a call to the constructor Vol{T<:Real}(...),
since type constructors fall back to convert methods.
while loading In[126], in expression starting on line 1

It seems like we can no longer construct an object of this type. In fact we can, by explicitly specifying the type:

In [127]:
Vol{Int}(3)

Vol{Int64}(3)

In [128]:
Vol{Float64}(3)

Vol{Float64}(3.0)

In [130]:
Vol{String}(3)

LoadError: LoadError: TypeError: Vol: in T, expected T<:Real, got Type{String}
while loading In[130], in expression starting on line 1

In [129]:
Vol(3)

LoadError: LoadError: MethodError: Cannot `convert` an object of type Int64 to an object of type Vol{T<:Real}
This may have arisen from a call to the constructor Vol{T<:Real}(...),
since type constructors fall back to convert methods.
while loading In[129], in expression starting on line 1

In [131]:
methods(Vol)

# 1 method for generic function "(::Type)":
(::Type{T}){T}(arg) at sysimg.jl:53

In [132]:
methods(Vol{Int})

# 2 methods for generic function "(::Type)":
(::Type{Vol{T}}){T<:Real}(x::Real) at In[125]:5
(::Type{T}){T}(arg) at sysimg.jl:53

Again, this is a pain and undesirable -- Julia should be able to infer the type for us.
The solution is to analyse what we want Julia to do, and then tell Julia to do exactly that. 

We would like Julia to look at the type of the object `x` passed to the constructor - call it `T` - and automatically use the constructor *for `Vol{T}`*, passing `x` as an argument, as we just did in the explicit call.

The syntax to do this looks strange at first (and second) glance:

In [133]:
Vol(x::T) = Vol{T}(x)

LoadError: LoadError: UndefVarError: T not defined
while loading In[133], in expression starting on line 1

In [ ]:
f(x::Int) = 2x

In [136]:
eltype{T}(x::T) = T

eltype (generic function with 1 method)

In [138]:
V = Vol{Int}(3)

Vol{Int64}(3)

In [139]:
eltype(V)

Vol{Int64}

In [ ]:
Vol{T}(x::T) = Vol{T}(x)

In [142]:
Vol{T}(x::T) = Vol{T}(x::T)

Vol{T<:Real}

Now it works:

In [144]:
@which 3//4

//(n::Integer, d::Integer) at rational.jl:22

In [143]:
Vol(3)

Vol{Int64}(3)

In [ ]:
methods(Vol)

What is going on here? We seem to almost be repeating ourselves, writing the same both on the left and right of the assignment operator `=`. Once again, we are telling Julia to treat `T` as a type parameter. However, the meaning of `{T}` is *different* on the two sides.

- The left-hand side means: "for **any** type `T`, define a function `Vol` that takes a parameter `x` *of type `T`*".
This can be thought of as a specification for a matching algorithm: look at the function call and see if it matches this template.


- The assignment operator has its usual meaning: "define the function on the left by the expression on the right".


- The right-hand side means: "call the constructor `Vol{T}` with argument `x`".

The point is that by the time the right-hand side is evaluated, Julia has **already realised** what `T` must be, since the function definition is matched by a call to `Vol` with an argument `x` of *some* type `T`, which Julia infers (i.e., it chooses `T` to match the type of the object that was passed in)!

The parametrised definition can be thought of as defining a "potentially infinite" number of different functions. One is instantiated, however, *only* if and when the function is actually called with an argument of that type.

Furthermore, Julia is sophisticated enough that we can make multiple definitions like this. For example, we may want to store all volumes that are integers as exact integers, but all floating-point volumes as `BigFloat`s. We can do this by making a special method for floats:

In [150]:
Vol{U<:AbstractFloat}(x::U) = Vol{BigFloat}(x)

Vol{T<:Real}

In [149]:
3.5::BigFloat

LoadError: LoadError: TypeError: typeassert: expected BigFloat, got Float64
while loading In[149], in expression starting on line 1

In [148]:
methods(Vol)

# 3 methods for generic function "(::Type)":
(::Type{Vol}){T<:AbstractFloat}(x::T) at In[145]:1
(::Type{Vol}){T}(x::T) at In[142]:1
(::Type{T}){T}(arg) at sysimg.jl:53

In [146]:
Vol(3.5)

Vol{BigFloat}(3.500000000000000000000000000000000000000000000000000000000000000000000000000000)

Passing in integers does *not* match this new definition, and so still uses the previous definition:

In [147]:
Vol(3)

Vol{Int64}(3)

In [ ]:
methods(Vol)

This is *multiple dispatch** in action: when the generic function `Vol` is called, the correct method is called based on matching the type of the argument `T`.

## Arithmetic: functions on parametric types

We would like to be able to do arithmetic on objects of our type: 

In [151]:
V1 = Vol(3)
V2 = Vol(4)

Vol{Int64}(4)

In [152]:
V1 + V2

LoadError: LoadError: MethodError: no method matching +(::Vol{Int64}, ::Vol{Int64})
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:138
while loading In[152], in expression starting on line 1

Since we have not specified what `+` means for objects of our type, Julia does not know what to do, so we must explicitly define `+` for objects of our type.

We may, for some reason, want to define the sum of volumes only if they contain objects of the same type. We can follow the idea and syntax discussed for the constructor:

In [154]:
+(V1::Vol{Int64}, V2::Vol{Int64}) = Vol{Int64}(V1.value + V2.value)

+ (generic function with 164 methods)

Here, foo exists only inside the for loop:

In [169]:
workspace()

In [171]:
type Vol{T}
    value::T
end

In [172]:
for T in (Int64, Float64, )
    foo(V1::Vol{T}, V2::Vol{T}) = Vol{T}(V1.value + V2.value)
    
    println(foo(Vol(3), Vol(4)))
end

Vol{Int64}(7)
Vol{Int64}(7)


In [174]:
for T in (Int64, Float64, )
    foo(V1::Vol{T}, V2::Vol{T}) = Vol{T}(V1.value + V2.value)
    
    println(foo(Vol(3), Vol(4)))
end

Vol{Int64}(7)
Vol{Int64}(7)


In [173]:
foo

foo (generic function with 2 methods)

In [166]:
for T in (Int64, Float64, )
    @eval foo(V1::Vol{T}, V2::Vol{T}) = Vol{T}(V1.value + V2.value)
end

In [168]:
methods(foo)

# 2 methods for generic function "foo":
foo(V1::Vol{Float64}, V2::Vol{Float64}) at In[166]:2
foo(V1::Vol{Int64}, V2::Vol{Int64}) at In[166]:2

In [160]:
methods(+)

# 165 methods for generic function "+":
+(x::Bool, z::Complex{Bool}) at complex.jl:136
+(x::Bool, y::Bool) at bool.jl:48
+(x::Bool) at bool.jl:45
+{T<:AbstractFloat}(x::Bool, y::T) at bool.jl:55
+(x::Bool, z::Complex) at complex.jl:143
+(x::Bool, A::AbstractArray{Bool,N<:Any}) at arraymath.jl:91
+(x::Float32, y::Float32) at float.jl:239
+(x::Float64, y::Float64) at float.jl:240
+(z::Complex{Bool}, x::Bool) at complex.jl:137
+(z::Complex{Bool}, x::Real) at complex.jl:151
+(a::Float16, b::Float16) at float16.jl:136
+(x::Char, y::Integer) at char.jl:40
+(c::BigInt, x::BigFloat) at mpfr.jl:240
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) at gmp.jl:298
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt) at gmp.jl:291
+(a::BigInt, b::BigInt, c::BigInt) at gmp.jl:285
+(x::BigInt, y::BigInt) at gmp.jl:255
+(x::BigInt, c::Union{UInt16,UInt32,UInt64,UInt8}) at gmp.jl:310
+(x::BigInt, c::Union{Int16,Int32,Int64,Int8}) at gmp.jl:326
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) at mpfr.jl:388
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) at mpfr.jl:381
+(a::BigFloat, b::BigFloat, c::BigFloat) at mpfr.jl:375
+(x::BigFloat, c::BigInt) at mpfr.jl:236
+(x::BigFloat, y::BigFloat) at mpfr.jl:205
+(x::BigFloat, c::Union{UInt16,UInt32,UInt64,UInt8}) at mpfr.jl:212
+(x::BigFloat, c::Union{Int16,Int32,Int64,Int8}) at mpfr.jl:220
+(x::BigFloat, c::Union{Float16,Float32,Float64}) at mpfr.jl:228
+{T}(B::BitArray{2}, J::UniformScaling{T}) at linalg/uniformscaling.jl:38
+(a::Base.Pkg.Resolve.VersionWeights.VWPreBuildItem, b::Base.Pkg.Resolve.VersionWeights.VWPreBuildItem) at pkg/resolve/versionweight.jl:85
+(a::Base.Pkg.Resolve.VersionWeights.VWPreBuild, b::Base.Pkg.Resolve.VersionWeights.VWPreBuild) at pkg/resolve/versionweight.jl:131
+(a::Base.Pkg.Resolve.VersionWeights.VersionWeight, b::Base.Pkg.Resolve.VersionWeights.VersionWeight) at pkg/resolve/versionweight.jl:185
+(a::Base.Pkg.Resolve.MaxSum.FieldValues.FieldValue, b::Base.Pkg.Resolve.MaxSum.FieldValues.FieldValue) at pkg/resolve/fieldvalue.jl:44
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.CompoundPeriod) at dates/periods.jl:314
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.Period) at dates/periods.jl:312
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.TimeType) at dates/periods.jl:359
+(dt::DateTime, z::Base.Dates.Month) at dates/arithmetic.jl:37
+(dt::DateTime, y::Base.Dates.Year) at dates/arithmetic.jl:13
+(x::DateTime, y::Base.Dates.Period) at dates/arithmetic.jl:64
+(x::Date, y::Base.Dates.Day) at dates/arithmetic.jl:62
+(x::Date, y::Base.Dates.Week) at dates/arithmetic.jl:60
+(dt::Date, z::Base.Dates.Month) at dates/arithmetic.jl:43
+(dt::Date, y::Base.Dates.Year) at dates/arithmetic.jl:17
+(V1::Vol{Int64}, V2::Vol{Int64}) at In[159]:2
+(V1::Vol{Float64}, V2::Vol{Float64}) at In[159]:2
+(y::AbstractFloat, x::Bool) at bool.jl:57
+{T<:Union{Int128,Int16,Int32,Int64,Int8,UInt128,UInt16,UInt32,UInt64,UInt8}}(x::T, y::T) at int.jl:32
+(x::Integer, y::Ptr) at pointer.jl:108
+(z::Complex, w::Complex) at complex.jl:125
+(z::Complex, x::Bool) at complex.jl:144
+(x::Real, z::Complex{Bool}) at complex.jl:150
+(x::Real, z::Complex) at complex.jl:162
+(z::Complex, x::Real) at complex.jl:163
+(x::Rational, y::Rational) at rational.jl:199
+(x::Integer, y::Char) at char.jl:41
+{N}(i::Integer, index::CartesianIndex{N}) at multidimensional.jl:58
+(c::Union{UInt16,UInt32,UInt64,UInt8}, x::BigInt) at gmp.jl:314
+(c::Union{Int16,Int32,Int64,Int8}, x::BigInt) at gmp.jl:327
+(c::Union{UInt16,UInt32,UInt64,UInt8}, x::BigFloat) at mpfr.jl:216
+(c::Union{Int16,Int32,Int64,Int8}, x::BigFloat) at mpfr.jl:224
+(c::Union{Float16,Float32,Float64}, x::BigFloat) at mpfr.jl:232
+(x::Irrational, y::Irrational) at irrationals.jl:88
+(x::Number) at operators.jl:115
+{T<:Number}(x::T, y::T) at promotion.jl:255
+(x::Number, y::Number) at promotion.jl:190
+(r1::OrdinalRange, r2::OrdinalRange) at operators.jl:505
+{T<:AbstractFloat}(r1::FloatRange{T}, r2::FloatRange{T}) at operators.jl:512
+{T<:

In [153]:
import Base.+

"""For any T inferred from my arguments, I will grab the T from 
the arguments and apply it all through the line to the very line.

This is really a kind of code generation where T is just replaced with the correct type.
"""
+(V1::Vol{T}, V2::Vol{T}) = Vol{T}(V1.value + V2.value)

LoadError: LoadError: UndefVarError: T not defined
while loading In[153], in expression starting on line 480

This looks a bit noisy, but we just have to parse it piece by piece: it is a method of `+`, that is parametric with one type parameter `T`, and whose arguments are `Vol`s parametrised by **the same** `T`. This is important, since it means that the definition **will not match** function calls with two different types. This is again an aspect of multiple dispatch. (Strictly speaking, the previous example was single dispatch, since a single argument was involved. Here it is multiple dispatch, since there are two arguments to the function.)

Again, we can think of the `{T}` on the left as "define a template function that works for any `T`, as follows", and the `{T}` on the left is to refer to the parametric type `Vol{T}` with the `T` that has already been inferred (assigned) from the arguments `V1` and `V2`:

In [ ]:
V1 + V2

Here, Julia examined `V1` and realised that it matches the definition when `T` is `Int64`.

However, this may be too restrictive; for example, we may wish to be able to add volumes with different integer types inside:

In [ ]:
Vol{Int64}(1) + Vol{BigInt}(3)

We can again define a more general method:

In [ ]:
+{T1<:Integer, T2<:Integer}(V1::Vol{T1}, V2::Vol{T2}) = Vol{promote_type(T1,T2)}(V1.value + V2.value)

[We could have left out the type parameter on the right-hand side, but this leads to a method ambiguity warning, in which two different methods match the combination of argument types, in this case when we have two equal integer types.]

Here, we have *two* type parameters that are both subtypes of (`<:` again) the abstract `Integer` type, and `promote_type(T,S)` is a function from `Base` that determines the smallest supertype of both `T` and `S` that is able to correctly represent values of both types. 

In [ ]:
Vol{Int64}(1) + Vol{BigInt}(3)

In [ ]:
Vol{Float64}(1) + Vol{BigFloat}(3)

Note that sometimes such a redefinition will apparently not work, since the previous definition, *even if it threw an error*, will be cached by Julia. In this case, it will be necessary to clear the workspace first and redefine everything. This can soon become tiresome and may be partially solved by moving the definitions out into a file, and `include`ing the file after calling `workspace()` to define the types. This kind of workflow discussion is best decided in practice. [In a future version of Julia, this caching of previous definitions will no longer apply; redefining a method will redefine all methods that depend on it.]

## Adding more parameters 

Types may be parametrized by more than one parameter. In the case of volumes, we should not be able to "add" a 1-dimensional volume (length) to a 2-dimensional one (area), since this makes no sense physically or mathematically; on the other hand, we can multiply them to get a 3-dimensional volume. We thus need to add an integer parameter to the `Vol` type:

In [ ]:
workspace()

In [ ]:
immutable Vol{N, T<:Real}
    value::T   
end

Again, a default constructor is created, but now we must explicitly specify the types, since the type `N` cannot be inferred from the argument:

In [ ]:
Vol{3, Float64}(3)

represents a three-dimensional volume of magnitude 3.

We wish to define the sum *only* when the dimensions are the same:

In [ ]:
import Base.+

+{N,T}(V1::Vol{N,T}, V2::Vol{N,T}) = Vol{N,T}(V1.value + V2.value)

This will match only when both arguments have the same parameters `T` and `N`. We could also promote the types:

In [ ]:
+{N,T1,T2}(V1::Vol{N,T1}, V2::Vol{N,T2}) = Vol{N,promote_type(T1,T2)}(V1.value + V2.value)

We make a useful outer constructor that can infer the type of its argument, but still needs an explicit dimension:

In [ ]:
Vol{T}(N, x::T) = Vol{N,T}(x)

In [ ]:
Vol(3, 4.5)

We would also like to display objects in a nicer way:

In [ ]:
import Base.show
show{N,T}(io::IO, x::Vol{N,T}) = print(io, N, "-dimensional volume with magnitude ", x.value)

In [ ]:
A = Vol(2, 3.0)   # volume of dimension 2 with value 3.0

In [ ]:
ℓ = Vol(1, 10.0)  # write as \ell<TAB>

In [ ]:
A + ℓ

It is not nice for the user to throw this slightly cryptic error message. We can provide a better error message when volumes of different dimension are added; we should check that volumes of the same dimension *still works*. Such things should be codified in unit test suites.

In [ ]:
workspace()

In [ ]:
immutable Vol{N,T<:Real}
    value::T   
end

Vol{T}(N, x::T) = Vol{N,T}(x)

import Base.+

+{N,T}(V1::Vol{N,T}, V2::Vol{N,T}) = Vol{N,T}(V1.value + V2.value)
+{N1,N2,T}(V1::Vol{N1,T}, V2::Vol{N2,T}) = throw(ArgumentError("Volumes of different dimension cannot be added"))

In [ ]:
V1 = Vol(3, 3)
V2 = Vol(3, 4)
V1 + V2

In [ ]:
Vol(3, 3) + Vol(2, 4)

**Exercise**
Define the product (`*`) of any two objects of type `Vol`.

## Conversion

We may want to define functions like `sin` that act on objects of our new type.
[We may *not* want to define such functions, on the other hand, since we could argue that a `Vol` is not a real number, but rather is a combination of a real number together with a dimension.]
Let's try it:

In [ ]:
V1

In [ ]:
sin(V1)

Of course, Julia doesn't know how to do this, since we haven't defined it.
However, an object of type `Vol` is just a simple wrapper around a number, so we could just say

In [ ]:
import Base.sin
sin(x::Vol) = sin(x.value)

In [ ]:
sin(V1)

It is painful, boring and error-prone to do this for every function (although this may be done using metaprogramming). An alternative is to tell Julia explicitly that a `Vol` is actually a kind of real number, i.e. a subtype of `Real`.

Note that although this is not mathematically correct, it is a convenience to simplify the code.

[Future versions of Julia are slated to include **traits** (also called **interfaces**) to allow this behavior without pretending that types are subsets of other types.

In [ ]:
workspace()

In [ ]:
immutable Vol{N,T<:Real} <: Real
    value::T   
end

In [ ]:
V = Vol{3,Float64}(3)

In [ ]:
sin(V)

We now get a different error. We can see which method Julia is now using:

In [ ]:
@which sin(V)

This is a generic "fallback" ("catch-all") method, which is called when no more specialised method has been defined.

Following the link to the Julia source code, which is written in Julia and therefore amazingly understandable (with some work), and if we understand some metaprogramming, we find that the definition of `sin(x::Real)` is

    sin(x::Real) = sin(float(x))

We could just define `float` for our type, which is supposed to convert an object to it's most natural floating-point equivalent:

In [ ]:
workspace()

In [ ]:
immutable Vol{N,T<:Real} <: Real
    value::T   
end

import Base.float
float(V::Vol) = float(V.value)

In [ ]:
V = Vol{3,Float64}(3)
sin(V)

However, in fact `float` is already defined for *any* `x::Real`:

In [ ]:
workspace()

In [ ]:
immutable Vol{N,T<:Real} <: Real
    value::T   
end

In [ ]:
V = Vol{3,Float64}(3)

In [ ]:
@which float(V)

In [ ]:
float(3)

The definition reads:

    float(x) = convert(AbstractFloat, x)

Finally, we see that rather we need to investigate Julia's capabilities for [conversion and promotion](http://docs.julialang.org/en/release-0.4/manual/conversion-and-promotion/) and define how to `convert` an object of our type into a float:

In [ ]:
import Base.convert
convert(::Type{AbstractFloat}, x::Vol) = float(x.value)

In [ ]:
convert(AbstractFloat, V)

Now any standard unary function works:

In [ ]:
sin(V), cos(V), exp(V)

`::Type{AbstractFloat}` refers to an argument which has type "type of `AbstractFloat`"; the only object that has that type is `AbstractFloat` itself; this thus matches function calls of the form 

    convert(AbstractFloat, V)

## Promotion 

In the case of `Vol`, it is not natural to define all arithmetic operations, so let's simplify even further and just wrap a `Float64` in a `Prob` type, representing a probability. [We will assume that it makes sense to define all arithmetic operations here, although in a given application that may not be the case.]

In [ ]:
workspace()

In [ ]:
immutable Prob{T<:AbstractFloat} <: Real
    _::T  # dummy name
    
    function Prob(x)
        (!(0 <= x <= 1)) && throw(ArgumentError("Probability must be between 0 and 1."))
        
        new(x)
    end
end

Prob{T<:AbstractFloat}(x::T) = Prob{T}(x)

[Note that `_` is a valid variable name in Julia, and is often used for a variable whose name we don't care about.]

In [ ]:
x = Prob(0.5)

We see that multiplying `x` by a real number does not give a `MethodError`:

In [ ]:
3x

since a `Vol` is a subtype of `Number`. 

Rather, Julia is trying to use its promotion machinery, in which such operations are handled by **promoting** both arguments to the same type:

In [ ]:
@which 3x

We can see the process using `@which` (or using the debugger in Julia v0.5):

In [ ]:
@which promote(3, x)

In [ ]:
@which promote_type(Int, Prob{Float64})

which is defined in terms of `promote_rule(T,S)`, e.g.

In [ ]:
@which promote_rule(Float64, Int)

So if we want to have promotion work, we need to define `promote_rule`:

In [ ]:
workspace()

In [ ]:
immutable Prob{T<:AbstractFloat} <: Real
    value::T
end

Prob{T<:AbstractFloat}(x::T) = Prob{T}(x)

import Base.*
*{T<:AbstractFloat}(x::Prob{T}, y::Prob{T}) = Prob{T}(x.value*y.value)

In [ ]:
p1 = Prob(0.3)
p2 = Prob(0.4)
p1 * p2

In [ ]:
0.3 * p2

In [ ]:
import Base: promote_rule, convert

promote_rule{S<:Real, T<:Real}(::Type{Prob{T}}, ::Type{S}) = Prob{promote_type(S, T)}

convert{S<:Real, T<:Real}(::Type{Prob{T}}, x::S) = Prob(float(x))
convert{T<:Real}(::Type{AbstractFloat}, x::Prob{T}) = x.value


In [ ]:
x = Prob(0.5)


In [ ]:
3x

In [ ]:
promote(3, x)

These rules are always tricky to get right!

## `Vector`s of objects of parametric type

Suppose we make a vector of rationals, which is a parametric type

In [ ]:
v = [3//4, 4//5]

In [ ]:
typeof(v)

A **typealias**, that is, an alternative (shorter and/or more intuitive) name for the type `Array{T,1}` is `Vector{T}`:

In [ ]:
Array{Rational{Int64},1} === Vector{Rational{Int}}  

Here, the `===` operator checks for identity (rather than just equality) of objects:

In [ ]:
1 == 1.0

In [ ]:
1 === 1.0

We also have a subtype relationship:

In [ ]:
Rational{Int} <: Rational

where `Rational` is an abstract type, meaning that we cannot create objects of type `Rational`; they must always be parametrised.

**However**, we have

In [ ]:
Vector{Rational{Int}} <: Vector{Rational}

Thus if we define a method

In [ ]:
h(x::Vector{Rational}) = 2x

that accepts arguments that are `Vector`s of `Rational`s, we have

In [ ]:
v = [3//4, 4//5]
h(v)

i.e. `h` **does not** accept arguments of type `Vector{Rational{Int}}`. Instead, we must parametrize explicitly:

In [ ]:
h{T}(x::Vector{Rational{T}}) = 2x

In [ ]:
h(v)

Here, the syntax `{T}` means "define a function `h` with this template, for *any* type `T`.

## Function wrappers (functors) and performance

Starting in Julia v0.5, functions are treated differently: the type of each function is unique:

In [175]:
typeof(sin)

Base.#sin

In [176]:
typeof(cos)

Base.#cos

In [177]:
typeof(x -> x)

##1#2

In [178]:
typeof(x -> x)

##3#4

These types are all subtypes of the abstract type `Function`:

In [ ]:
supertype(typeof(sin)), supertype(typeof(x->x))

In [179]:
typeof(sin) <: Function

true

This allows us significant performance gains when functions are passed as parameters to other functions, such as `map`. (These latter functions are then called ["higher-order functions"](https://en.wikipedia.org/wiki/Higher-order_function).) It also allows performance gains when functions are wrapped in user-defined types that are supposed to behave like functions (["function objects"](https://en.wikipedia.org/wiki/Function_object), sometimes called "functors"). 

We might want to write the following to wrap a function and a value to call in a single object:

In [180]:
workspace()

In [181]:
type Wrapper1
    f::Function
    x::Float64
end

In [204]:
w = Wrapper1(sin, 10)

Wrapper1(sin,10.0)

We want to think of `w` itself as a function, but currently it cannot be called:

In [183]:
w()

LoadError: LoadError: MethodError: objects of type Wrapper1 are not callable
while loading In[183], in expression starting on line 1

We can define this as follows:

In [184]:
v = [1, 2]

2-element Array{Int64,1}:
 1
 2

In [188]:
(v::Array)(i) = v[i+1]

In [191]:
v(0)

1

In [192]:
v(1)

2

In [203]:
(w::Wrapper1)() = w.f(w.x)

In [195]:
w(), sin(10)

(-0.5440211108893698,-0.5440211108893698)

However, this is not performant. The main reason is that we could now do the following:

In [196]:
w.f = cos

cos (generic function with 10 methods)

In [197]:
w(), cos(10)

(-0.8390715290764524,-0.8390715290764524)

It is only *at run time*, when the function call is executed, that Julia knows which function is stored inside the object, and at that moment it must do a (slow) look-up.

In [210]:
type IntFunc
    x::Int
end

In [214]:
(X::IntFunc)(y::Int) = X.x*y

In [215]:
x = IntFunc(3)

IntFunc(3)

In [216]:
x(10)

30

In [217]:
T = typeof(sin)
Wrapper2{T}(cos, 10)

LoadError: LoadError: MethodError: Cannot `convert` an object of type Base.#cos to an object of type Base.#sin
This may have arisen from a call to the constructor Base.#sin(...),
since type constructors fall back to convert methods.
while loading In[217], in expression starting on line 2

In [225]:
for f in (sin, cos, tan)
    T = typeof(f)
    @eval (w::Wrapper2{T})() = (w.f)(w.x)
end

In [226]:
methods(Wrapper2)

# 2 methods for generic function "(::Type)":
(::Type{Wrapper2}){F}(f::F, x::Float64) at In[198]:2
(::Type{T}){T}(arg) at sysimg.jl:53

In [ ]:
(w::Wrapper2{F}){F}() = w.f(w.x)

To solve this problem, we parameterize the wrapper type:

In [198]:
type Wrapper2{F}
    f::F
    x::Float64
end

In [205]:
w2 = Wrapper2(sin, 10.0)

Wrapper2{Base.#sin}(sin,10.0)

Again, Julia infers the value of `F`, and creates an object of this new type, with a parameter `F` that is the type of `sin`. We again define how to call an object of this type, now with a slightly more complicated syntax, due to the parameter:

In [ ]:
+{T}(x::T, y::T) = ...

In [222]:
(w::Wrapper2{F}){F}() = w.f(w.x)

In [206]:
@code_llvm w()


define %jl_value_t* @julia_Wrapper1_72510(%jl_value_t*, %jl_value_t**, i32) #0 {
top:
  %3 = alloca %jl_value_t**, align 8
  store volatile %jl_value_t** %1, %jl_value_t*** %3, align 8
  %4 = call %jl_value_t*** @jl_get_ptls_states() #1
  %5 = alloca [4 x %jl_value_t*], align 8
  %.sub = getelementptr inbounds [4 x %jl_value_t*], [4 x %jl_value_t*]* %5, i64 0, i64 0
  %6 = getelementptr [4 x %jl_value_t*], [4 x %jl_value_t*]* %5, i64 0, i64 2
  %7 = bitcast %jl_value_t** %6 to i8*
  call void @llvm.memset.p0i8.i32(i8* %7, i8 0, i32 16, i32 8, i1 false)
  %8 = bitcast [4 x %jl_value_t*]* %5 to i64*
  store i64 4, i64* %8, align 8
  %9 = bitcast %jl_value_t*** %4 to i64*
  %10 = load i64, i64* %9, align 8
  %11 = getelementptr [4 x %jl_value_t*], [4 x %jl_value_t*]* %5, i64 0, i64 1
  %12 = bitcast %jl_value_t** %11 to i64*
  store i64 %10, i64* %12, align 8
  store %jl_value_t** %.sub, %jl_value_t*** %4, align 8
  %13 = getelementptr [4 x %jl_value_t*], [4 x %jl_value_t*]* %5, i64 0, i

In [207]:
@code_llvm w2()


define double @julia_Wrapper2_72500(%jl_value_t*) #0 {
top:
  %1 = bitcast %jl_value_t* %0 to double*
  %2 = load double, double* %1, align 16
  %3 = call double @julia_sin_72477(double %2) #0
  ret double %3
}


In [ ]:
@code_llvm 

The difference in performance can be measured. The dynamic look-up can be seen using `@code_llvm` or `@code_native`.

For an interesting discussion about how this is actually used to create very efficient complicated array structures, see [Tim Holy's keynote at JuliaCon 2016](https://www.youtube.com/watch?v=fl0g9tHeghA).

One caveat (trade-off) is that this approach requires a new version of `Wrapper2` to be compiled for each function that is passed to it.

## Abstract types and inheritance

A type of "inheritance" is possible in Julia. As a simple example, consider two types of objects in a simulation, particles, discs, and squares. These are all subtypes of objects that will have certain properties in common, for example that they can move. Each type of object has a position, but discs and squares also have other properties. 

We make an abstract type `MovableObject` and subtypes of it: 

In [99]:
abstract MovableObject

immutable Particle <: MovableObject
    pos::Int
end

immutable Disc <: MovableObject
    pos::Int
    radius::Float64
end

immutable Square <: MovableObject
    pos::Int
    side::Float64
end

We could imagine that most objects will move by changing their position:

In [101]:
move(obj::MovableObject) = obj.pos += 1

move (generic function with 1 method)

but that `Square`s expand and contract by changing their side length

In [102]:
move(s::Square) = s.side += 1

move (generic function with 2 methods)

The `move` function applies to any `MovableObject`, but the specialised version of the function for `Square`s automatically takes priority.